This notebook is for comparing the gap filling techniques visually

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
import torch

In [2]:
# First import the model
from modules.models import MLP
from modules.util import grab_data

In [3]:
# Load the data on which predictions and or gap filling are to be done
data, labels, dim_in, dim_out = grab_data('data/data_files_modified/GW_fluxes_30min_20240401_20240608_mod.csv', return_dataset = False )

In [4]:
# Load the model
model = MLP(dim_in, dim_out, num_hidden_units=30, num_hidden_layers=4)
model.load_state_dict(torch.load('model_saves/mlp_1.pth'))
model.eval()

MLP(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=8, out_features=30, bias=True)
    (1): ReLU()
    (2): Linear(in_features=30, out_features=30, bias=True)
    (3): ReLU()
    (4): Linear(in_features=30, out_features=30, bias=True)
    (5): ReLU()
    (6): Linear(in_features=30, out_features=30, bias=True)
    (7): ReLU()
    (8): Linear(in_features=30, out_features=30, bias=True)
    (9): ReLU()
    (10): Linear(in_features=30, out_features=2, bias=True)
  )
)